<a href="https://colab.research.google.com/github/Csrul0/ESTRUCTURA_DE_DATOS_Y_SU_PROCESAMIENTO_EQP5/blob/main/Ev2_Equipo_5_Estructura_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import datetime
import random
import re
import pandas as pd
import os
import csv

# Crear una lista para almacenar las notas
notas = []

# Contador global para generar folios únicos
contador_folios = 1

# Diccionario para almacenar las notas canceladas
notas_canceladas = {}

fecha_actual = datetime.date.today()

email_pattern =  r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'

def obtener_homoclave():
    caracteres = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    return ''.join(random.sample(caracteres, 3))

def encontrar_primera_vocal(cadena):
    vocales = "AEIOU"
    for letra in cadena:
        if letra in vocales:
            return letra
    return ""

def guardar_estado():
    with open('estado_app.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        # Guardar las notas
        for nota in notas:
            writer.writerow([nota['folio'], nota['fecha'], nota['nombre'], nota['RFC'], nota['Correo electronico'], nota['detalle'], nota['monto_total'], nota['cancelada']])
        # Guardar las notas canceladas
        for folio, nota in notas_canceladas.items():
            writer.writerow([folio, nota['fecha'], nota['nombre'], nota['RFC'], nota['Correo electronico'], nota['detalle'], nota['monto_total'], nota['cancelada']])

# Función para cargar el estado desde un archivo CSV
def cargar_estado():
    if os.path.isfile('estado_app.csv'):
        with open('estado_app.csv', mode='r') as file:
            reader = csv.reader(file)
            for row in reader:
                folio, fecha_str, nombre, rfc, email, detalle_str, monto_total, cancelada = row
                fecha = datetime.datetime.strptime(fecha_str, '%Y-%m-%d').date()
                detalle = eval(detalle_str)  # Convierte la representación de cadena de lista a lista real
                nota = {
                    "folio": int(folio),
                    "fecha": fecha,
                    "nombre": nombre,
                    "RFC": rfc,
                    "Correo electronico": email,
                    "detalle": detalle,
                    "monto_total": float(monto_total),
                    "cancelada": bool(cancelada)
                }
                if nota["cancelada"]:
                    notas_canceladas[nota["folio"]] = nota
                else:
                    notas.append(nota)
    else:
        print("No se encontró un archivo de estado anterior. Se parte de un estado inicial vacío.")

# Cargar el estado al inicio del programa
cargar_estado()

while True:
    print("MENU PRINCIPAL:")
    print("1. Registrar una nota")
    print("2. Consultar y Reportes")
    print("3. Cancelar una nota")
    print("4. Recuperar una nota")
    print("5. Salir")

    opcion = input("Escoja una opción (1/2/3/4/5): ")

    while opcion not in ["1", "2", "3", "4", "5"]:
        print("Opción no válida. Por favor, seleccione una opción válida (1/2/3/4/5).")
        opcion = input("Escoja una opción (1/2/3/4/5): ")

    if opcion == "1":
        print("Ha escogido Registrar una nota")
        nombre = input("Ingresa tu nombre completo: ").upper()

        while not nombre.strip() or any(char.isdigit() for char in nombre):
            if not nombre.strip():
                print("El nombre no puede estar vacío.")
            elif any(char.isdigit() for char in nombre):
                print("El nombre no puede contener números.")
            nombre = input("Ingresa tu nombre completo: ").upper()


        apellidos = input("Ingresa tus apellidos paterno y materno, separados por un espacio: ").upper()
        while not apellidos.strip() or any(char.isdigit() for char in apellidos):
            if not apellidos.strip():
                print("Los apellidos no puede estar vacío.")
            elif any(char.isdigit() for char in apellidos):
                print("Los apellidos no puede contener números.")
            nombre = input("Ingresa tus apellidos: ").upper()

        fecha = input("Ingrese la fecha (dd/mm/aaaa): ")

        try:
            fecha_nota = datetime.datetime.strptime(fecha, '%d/%m/%Y').date()
            if fecha_nota > fecha_actual:
                print("La fecha no puede ser posterior a la fecha actual.")
                continue
        except ValueError:
            print("Formato de fecha incorrecto. Use dd/mm/aaaa.")
            continue

        while True:
          email_usuario = input("Ingrese su correo electronico:\n ")
          if re.match(email_pattern, email_usuario):
            print(f"Correo electronico aceptado: {email_usuario}")
            break
          else:
            print("Tu correo electronico no es vaido. Intente de nuevo ingresando un correo valido")

        fecha_nacimiento = input("Ingresa tu fecha de nacimiento (AAAA-MM-DD): ")

        apellidos = apellidos.split()
        inicial_apellido_paterno = apellidos[0][0]
        vocal_apellido_paterno = encontrar_primera_vocal(apellidos[0])
        inicial_apellido_materno = apellidos[1][0]
        inicial_nombre = nombre.split()[0][0]


        año_nacimiento = fecha_nacimiento.split('-')[0]
        mes_nacimiento = fecha_nacimiento.split('-')[1]
        dia_nacimiento = fecha_nacimiento.split('-')[2]


        homoclave = obtener_homoclave()


        rfc = (
            inicial_apellido_paterno +
            vocal_apellido_paterno +
            inicial_apellido_materno +
            inicial_nombre +
            año_nacimiento[2:] +
            mes_nacimiento +
            dia_nacimiento +
            homoclave
        )

        detalle = []
        while True:
            print("Ingrese el servicio libre (o presione Enter para terminar):")
            servicio_libre = input()
            if not servicio_libre:
                if not detalle:
                    print("Debe ingresar al menos un servicio.")
                    continue
                else:
                    break
            monto_servicio = input("Ingrese el monto del servicio libre: $")

            try:
                monto_servicio = float(monto_servicio)
                if monto_servicio <= 0:
                    print("El monto debe ser mayor que 0.")
                    continue
            except ValueError:
                print("El monto debe ser un número decimal válido.")
                continue

            detalle.append({"nombre": servicio_libre, "precio": monto_servicio})

        folio = contador_folios
        contador_folios += 1
        monto_total = sum(servicio["precio"] for servicio in detalle)
        nota = {
            "folio": folio,
            "fecha": fecha_nota,
            "nombre": nombre,
            "RFC": rfc,
            "Correo electronico": email_usuario,
            "detalle": detalle,
            "monto_total": monto_total,
            "cancelada": False  # Inicialmente la nota no está cancelada
        }
        notas.append(nota)
        print(f"Su folio es: {folio}")
        print(f"La fecha: {fecha_nota}")
        print(f"Su nombre es: {nombre}")
        print(f"Su RFC es: {rfc}")
        print(f"Su correo es: {email_usuario}")
        print("Detalle de la nota:")
        for servicio in detalle:
            print(f"- {servicio['nombre']}: ${servicio['precio']}")
        print(f"Monto total a pagar: ${monto_total}")

    elif opcion == "2":
        print("Ha escogido Consultar y Reportes")
        while True:
            print("SUBMENÚ DE CONSULTAS Y REPORTES:")
            print("1. Consulta por período")
            print("2. Consulta por folio")
            print("3. Consulta por cliente")
            print("4. Volver al menú principal")

            subopcion = input("Escoja una opción (1/2/3/4): ")

            while subopcion not in ["1", "2", "3", "4"]:
                print("Opción no válida. Por favor, seleccione una opción válida (1/2/3/4).")
                subopcion = input("Escoja una opción (1/2/3/4): ")

            if subopcion == "1":
                print("Ha escogido Consulta por período")
                fecha_inicio = input("Dame la fecha de Inicio que sea en este formato: dd/mm/aaaa: \n")

                while True:
                    try:
                        fecha_inicioP = datetime.datetime.strptime(fecha_inicio, '%d/%m/%Y').date()
                        if fecha_inicioP > fecha_actual:
                            print("La fecha de inicio no puede ser posterior a la fecha actual.")
                            fecha_inicio = input("Dame la fecha de Inicio que sea en este formato: dd/mm/aaaa: \n")
                        else:
                            break
                    except ValueError:
                        print("Formato de fecha incorrecto. Use dd/mm/aaaa.")
                        fecha_inicio = input("Dame la fecha de Inicio que sea en este formato: dd/mm/aaaa: \n")

                fecha_final = input("Dame la fecha de Final que sea en este formato: dd/mm/aaaa: \n")

                while True:
                    try:
                        fecha_finalP = datetime.datetime.strptime(fecha_final, '%d/%m/%Y').date()
                        if fecha_finalP > fecha_actual:
                            print("La fecha de final no puede ser posterior a la fecha actual.")
                            fecha_final = input("Dame la fecha de Final que sea en este formato: dd/mm/aaaa: \n")
                        else:
                            break
                    except ValueError:
                        print("Formato de fecha incorrecto. Use dd/mm/aaaa.")
                        fecha_final = input("Dame la fecha de Final que sea en este formato: dd/mm/aaaa: \n")

                notas_encontradas = []

                for nota in notas:
                    if fecha_inicioP <= nota["fecha"] <= fecha_finalP:
                        notas_encontradas.append(nota)

                if notas_encontradas:
                    print("Se encontraron notas para las fechas especificadas:")
                    print("FOLIO\tCLIENTE\t\tFECHA\tMONTO TOTAL")
                    for nota in notas_encontradas:
                        print(f"{nota['folio']}:\t{nota['nombre']}\t{nota['fecha']}\t${nota['monto_total']}")
                else:
                    print("No se encontraron notas para las fechas especificadas.")

            elif subopcion == "2":
                print("Ha escogido Consulta por folio")
                folio = input("Ingrese el folio de la nota a consultar: ")

                while True:
                    try:
                        folio = int(folio)
                        break
                    except ValueError:
                        print("El folio debe ser un número entero.")
                        folio = input("Ingrese el folio de la nota a consultar: ")

                nota_encontrada = None

                for nota in notas:
                    if nota["folio"] == folio:
                        nota_encontrada = nota
                        break

                if nota_encontrada:
                    if nota_encontrada["cancelada"]:
                        print("Esta nota ha sido cancelada.")
                    else:
                        print("\nFOLIO\tCLIENTE\t\tFECHA\tMONTO TOTAL")
                        print(f"{folio}:\t{nota_encontrada['nombre']}\t{nota_encontrada['fecha']}\t${nota_encontrada['monto_total']}")
                        print("Detalle de la nota:")
                        for servicio in nota_encontrada["detalle"]:
                            print(f"- {servicio['nombre']}: ${servicio['precio']}")
                else:
                    print("La nota no existe en el sistema o ha sido cancelada.")

            elif subopcion == "3":
                print("Ha escogido Consulta por cliente")

                # Crear un diccionario para almacenar los RFCs y sus folios correspondientes
                rfc_folios = {}

                # Crear una lista de RFCs únicos
                rfc_unicos = set(nota["RFC"] for nota in notas)

                # Asignar folios consecutivos a los RFCs
                for i, rfc in enumerate(rfc_unicos, start=1):
                    rfc_folios[i] = rfc

                # Mostrar la lista de RFCs ordenados alfabéticamente con sus folios
                print("RFCs disponibles:")
                for folio, rfc in rfc_folios.items():
                    print(f"{folio}. {rfc}")

                # Solicitar al usuario que elija un RFC por su folio
                folio_rfc = input("Ingrese el folio del RFC a consultar: ")

                while True:
                    try:
                        folio_rfc = int(folio_rfc)
                        if folio_rfc in rfc_folios:
                            break
                        else:
                            print("El folio ingresado no es válido.")
                            folio_rfc = input("Ingrese el folio del RFC a consultar: ")
                    except ValueError:
                        print("El folio debe ser un número entero.")
                        folio_rfc = input("Ingrese el folio del RFC a consultar: ")

                # Verificar si el folio es válido
                if folio_rfc in rfc_folios:
                    rfc_consultado = rfc_folios[folio_rfc]

                    # Filtrar notas por el RFC consultado
                    notas_cliente = [nota for nota in notas if nota["RFC"] == rfc_consultado]

                    if notas_cliente:
                        print(f"Notas para el RFC {rfc_consultado}:")

                        # Calcular el monto promedio de las notas del cliente consultado
                        monto_promedio = sum(nota["monto_total"] for nota in notas_cliente) / len(notas_cliente)
                        print(f"Monto promedio de las notas: ${monto_promedio:.2f}")

                        # Mostrar información de las notas del cliente
                        for nota in notas_cliente:
                            print("\nFOLIO\tFECHA\tMONTO TOTAL")
                            print(f"{nota['folio']}\t{nota['fecha']}\t${nota['monto_total']}")
                            print("Detalle de la nota:")
                            for servicio in nota["detalle"]:
                                print(f"- {servicio['nombre']}: ${servicio['precio']}")

                        # Preguntar si el usuario desea exportar la información a un archivo Excel
                        exportar_excel = input("¿Desea exportar esta información a un archivo Excel? (Sí/No): ").strip().lower()
                        if exportar_excel == "si":
                            # Crear un DataFrame con los datos de las notas del cliente
                            df = pd.DataFrame(notas_cliente)

                            # Crear un nombre de archivo con el RFC del cliente y la fecha actual
                            nombre_archivo = f"{rfc_consultado}_{fecha_actual}.xlsx"

                            # Guardar el DataFrame en un archivo Excel
                            df.to_excel(nombre_archivo, index=False)

                            print(f"Archivo Excel guardado en: {nombre_archivo}")

                    else:
                        print(f"No se encontraron notas para el RFC {rfc_consultado}.")

                else:
                    print("El folio ingresado no es válido.")

            elif subopcion == "4":
                break

    elif opcion == "3":
        print("Ha escogido Cancelar una nota")
        folio_cancelar = input("Ingrese el folio de la nota a cancelar: ")

        while True:
            try:
                folio_cancelar = int(folio_cancelar)
                break
            except ValueError:
                print("El folio debe ser un número entero.")
                folio_cancelar = input("Ingrese el folio de la nota a cancelar: ")

        nota_encontrada = None

        for nota in notas:
            if nota["folio"] == folio_cancelar:
                nota_encontrada = nota
                break

        if nota_encontrada:
            if nota_encontrada["cancelada"]:
                print("Esta nota ya ha sido cancelada anteriormente.")
            else:
                print("Datos de la nota a cancelar:")
                print("\nFOLIO\tCLIENTE\t\tFECHA\tMONTO TOTAL")
                print(f"{folio_cancelar}:\t{nota_encontrada['nombre']}\t{nota_encontrada['fecha']}\t${nota_encontrada['monto_total']}")
                confirmacion = input("¿Desea cancelar esta nota? (Sí/No): ").strip().lower()
                if confirmacion == "si":
                    nota_encontrada["cancelada"] = True
                    notas_canceladas[folio_cancelar] = nota_encontrada
                    print("La nota ha sido cancelada exitosamente.")
                else:
                    print("La nota no ha sido cancelada.")
        else:
            print("El folio ingresado no existe en el sistema o corresponde a una nota cancelada.")

    elif opcion == "4":
        print("Ha escogido Recuperar una nota")
        if notas_canceladas:
            print("Notas canceladas disponibles:")
            print("FOLIO\tCLIENTE\t\tFECHA\tMONTO TOTAL")
            for folio, nota in notas_canceladas.items():
                print(f"{folio}:\t{nota['nombre']}\t{nota['fecha']}\t${nota['monto_total']}")
            folio_recuperar = input("Ingrese el folio de la nota a recuperar (0 para cancelar): ")

            while True:
                try:
                    folio_recuperar = int(folio_recuperar)
                    break
                except ValueError:
                    print("El folio debe ser un número entero.")
                    folio_recuperar = input("Ingrese el folio de la nota a recuperar (0 para cancelar): ")

            if folio_recuperar != 0:
                nota_recuperar = notas_canceladas.get(folio_recuperar)
                if nota_recuperar:
                    print("Detalle de la nota a recuperar:")
                    print("\nFOLIO\tCLIENTE\t\tFECHA\tMONTO TOTAL")
                    print(f"{folio_recuperar}:\t{nota_recuperar['nombre']}\t{nota_recuperar['fecha']}\t${nota_recuperar['monto_total']}")
                    confirmacion = input("¿Desea recuperar esta nota? (Sí/No): ").strip().lower()
                    if confirmacion == "si":
                        nota_recuperar["cancelada"] = False
                        notas.append(nota_recuperar)
                        del notas_canceladas[folio_recuperar]
                        print("La nota ha sido recuperada exitosamente.")
                    else:
                        print("La nota no ha sido recuperada.")
                else:
                    print("El folio ingresado no corresponde a una nota cancelada.")
            else:
                print("Operación cancelada.")
        else:
            print("No hay notas canceladas disponibles.")

    elif opcion == "5":
        print("Ha escogido Salir")
        guardar_estado()
        print("¡Gracias por usar el programa de notas! Hasta luego.")
        break